In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt  # важно!

# Генерация синтетических данных
X = np.random.rand(1000, 10)
y = X.sum(axis=1) + np.random.normal(0, 0.1, size=1000)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Функция для построения модели с гиперпараметрами
def build_model(hp):
    model = keras.Sequential()

    # Первый слой
    model.add(layers.Dense(
        units=hp.Int("units_input", min_value=32, max_value=128, step=32),
        activation=hp.Choice("activation", ["relu", "tanh"]),
        input_shape=(X_train.shape[1],)
    ))

    # Дополнительные скрытые слои (до 2-х)
    for i in range(hp.Int("n_hidden", 0, 2)):
        model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=32, max_value=128, step=32),
            activation=hp.Choice("activation_hidden", ["relu", "tanh"])
        ))

    model.add(layers.Dense(1))  # Выход для регрессии

    # Компиляция
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")
        ),
        loss="mse",
        metrics=["mae"]
    )
    return model

# Настройка поиска
tuner = kt.RandomSearch(
    build_model,
    objective="val_loss",
    max_trials=20,
    executions_per_trial=1,
    directory="keras_tuner_dir",
    project_name="fcnn_regression"
)

# Запуск поиска
tuner.search(
    X_train, y_train,
    epochs=20,
    validation_data=(X_test, y_test),
    batch_size=32,
    verbose=1
)

# Получение лучшей модели и вывод параметров
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hp)

# Финальное обучение и оценка
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), verbose=1)
mse, mae = model.evaluate(X_test, y_test, verbose=1)

print("\n📊 Лучшие гиперпараметры:")
for key, val in best_hp.values.items():
    print(f"{key}: {val}")

print(f"\n📈 MSE на тесте: {mse:.4f}")
print(f"📈 MAE на тесте: {mae:.4f}")

Trial 2 Complete [00h 00m 14s]
val_loss: 0.04874005913734436

Best val_loss So Far: 0.047583695501089096
Total elapsed time: 00h 00m 40s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
32                |64                |units_input
relu              |relu              |activation
1                 |2                 |n_hidden
0.00015596        |0.0053482         |learning_rate
32                |32                |units_0
relu              |relu              |activation_hidden
32                |32                |units_1

Epoch 1/20
14/25 [===============>..............] - ETA: 0s - loss: 24.8032 - mae: 4.8803 

KeyboardInterrupt: 